In [10]:
import openai
from collections import defaultdict
from itertools import zip_longest
import re
import random
import matplotlib.pyplot as plt

In [11]:
with open('openAI.key', 'r') as file:
    openAI_key = file.read()

In [12]:
from openai import OpenAI
openai.api_key = ""#todo remove or make it read from a doc that I dont submit.
client = OpenAI(
    api_key=openAI_key,
)

In [13]:
model="gpt-3.5-turbo"#using this one cause gpt4 is expensive.

In [14]:
def lower_than_rule(number):
    return number < 50


In [44]:
import string
def generate_example_list(rule,example_generator,size=20,range_min=1,range_max=1000):
    random_example_list = []
    for i in range(size):
        random_example_list.append(example_generator(random_example_list,range_min=range_min,range_max=range_max))
    example_list=""
    for number in random_example_list:
        if example_list !="":
            example_list+=", "
        rule_value= lower_than_rule(number)
        example_list+=(f"{number} -> {rule_value}")
    return example_list,random_example_list


In [66]:
def generate_example_int(excluded_ints,range_min=1,range_max=1000):
    all_integers = list(range(range_min, range_max+1))
    valid_integers = [x for x in all_integers if x not in excluded_ints]
    test_number= random.choice(valid_integers)
    return random.randint(range_min, range_max)

In [70]:
def generate_example_int_or_str(excluded_ints,range_min=1,range_max=1000):
    all_integers = list(range(range_min, range_max+1))
    valid_integers = [x for x in all_integers if x not in excluded_ints]
    test_number= random.choice(valid_integers)
    string_or_int=[random.randint(range_min, range_max),random.choice(string.ascii_uppercase)]
    return random.choice(string_or_int)

In [71]:
def generate_prompt(rule,example_generator,example_list,excluded_ints,range_min=1,range_max=1000):
    test_number=example_generator(excluded_ints,range_min=range_min,range_max=range_max)
    test_number_rule_value=rule(test_number)
    prompt =example_list+f", {test_number}->"
    return prompt,test_number,test_number_rule_value


In [72]:
def test_examples(example_list,example_generator,existing_ints,rule,attempts=10,range_min=1,range_max=1000):
    correct_counter=0
    for i in range(attempts):
        prompt,test_number,test_number_rule_value=generate_prompt(rule,example_generator,example_list,existing_ints,range_min,range_max)
        message={"role": "user", "content": prompt}
        print("generating response....")#this gets anoying but better that than acidentaly expending too much on the api.
        chat_completion = client.chat.completions.create(model=model, temperature=1, messages=[message],max_tokens=1)
        response = chat_completion.choices[0].message.content
        if response == str(test_number_rule_value):#could fail due to tokenization weirdness but seems to work in practice
            correct_counter+=1 
        else:
            print(f"error on {test_number} expected {test_number_rule_value} returned {response}")
            
                
    print(f"Accuracy:{(correct_counter/attempts)*100}%")      

In [92]:
def generate_explanations(example_list,example_generator,rule,existing_ints,n_completions=5):
    prompt,test_number,test_number_rule_value=generate_prompt(rule,example_generator,example_list,existing_ints)
    test_mesage={"role": "user", "content":prompt}
    print("generating response....")
    chat_completion = client.chat.completions.create(model=model, temperature=1, messages=[test_mesage],max_tokens=1)
    response = chat_completion.choices[0].message.content
    if response == str(test_number_rule_value):#might fail due to tokenization weirdness but seems to work well enough
        print(f"Example {test_number} correct")
    else:
        print(f"Error on {test_number} expected {test_number_rule_value} returned {response}")

    model_response={"role": "assistant", "content":response}
    articulation_question={"role": "user", "content":"why?" }

    chat_completion = client.chat.completions.create(model=model, temperature=1, messages=[test_mesage,model_response,articulation_question],max_tokens=200,n=n_completions)
    print("Explanations generated")
    return chat_completion

In [77]:
example_list,existing_ints=generate_example_list(lower_than_rule,generate_example_int,40,range_max=100)
print(f"example_list:{example_list}")


example_list:47 -> True, 13 -> True, 1 -> True, 84 -> False, 90 -> False, 5 -> True, 21 -> True, 74 -> False, 50 -> False, 5 -> True, 47 -> True, 71 -> False, 23 -> True, 79 -> False, 21 -> True, 2 -> True, 65 -> False, 2 -> True, 3 -> True, 95 -> False, 72 -> False, 21 -> True, 77 -> False, 26 -> True, 2 -> True, 32 -> True, 91 -> False, 89 -> False, 16 -> True, 89 -> False, 88 -> False, 17 -> True, 16 -> True, 45 -> True, 7 -> True, 9 -> True, 4 -> True, 3 -> True, 84 -> False, 26 -> True


In [78]:
test_examples(example_list,generate_example_int,existing_ints,lower_than_rule,attempts=10,range_max=100)

generating response....
generating response....
generating response....
generating response....
error on 40 expected True returned False
generating response....
generating response....
generating response....
generating response....
generating response....
generating response....
Accuracy:90.0%


In [93]:
chat_completion=generate_explanations(example_list,generate_example_int,lower_than_rule,existing_ints)

generating response....
Example 854 correct
Explanations generated


In [94]:
for response in chat_completion.choices:
       print(response .message.content)

According to the given pattern, the number 854 does not have a specific "True" or "False" result associated with it.
I apologize for the incorrect answer. In order to determine if a number is true or false, we need more information or context. Could you please provide more information or clarify the criteria for determining if a number is true or false?
The given number 854 is not mentioned in the sequence of numbers provided.
I apologize for the mistake in my previous response. The number 854 is not included in the list you provided.
Sorry for the mistake. 854 is not provided in the given list, so I cannot determine if it is true or false.


In [95]:
def is_even_rule(number):
    return number % 2 == 0

In [96]:
example_list,existing_ints=generate_example_list(is_even_rule,generate_example_int,40,range_max=100)
print(f"example_list:{example_list}")
test_examples(example_list,generate_example_int,existing_ints,is_even_rule,attempts=10)

example_list:3 -> True, 86 -> False, 53 -> False, 60 -> False, 100 -> False, 18 -> True, 68 -> False, 16 -> True, 19 -> True, 48 -> True, 48 -> True, 56 -> False, 71 -> False, 28 -> True, 88 -> False, 49 -> True, 17 -> True, 75 -> False, 85 -> False, 83 -> False, 84 -> False, 53 -> False, 49 -> True, 1 -> True, 41 -> True, 78 -> False, 49 -> True, 22 -> True, 88 -> False, 14 -> True, 22 -> True, 78 -> False, 35 -> True, 27 -> True, 10 -> True, 9 -> True, 40 -> True, 4 -> True, 95 -> False, 33 -> True
generating response....
generating response....
error on 506 expected True returned False
generating response....
error on 952 expected True returned False
generating response....
error on 72 expected True returned False
generating response....
generating response....
generating response....
generating response....
generating response....
error on 464 expected True returned False
generating response....
error on 254 expected True returned False
Accuracy:50.0%


In [97]:
chat_completion=generate_explanations(example_list,generate_example_int,is_even_rule,existing_ints)
for response in chat_completion.choices:
       print(response .message.content)

generating response....
Example 647 correct
Explanations generated
I apologize for the mistake in my previous response. Based on the given list of numbers, the number 647 should be evaluated separately. Without any additional information or context, I cannot determine whether or not 647 is true or false.
The number 647 is not mentioned in the given list, so its truth value cannot be determined.
The number 647 is not divisible by 3.
Since there is no information given about the number 647, it is not possible to determine whether it is true or false.
Because 647 is not a multiple of 3.


In [98]:
def is_prime_rule(number):
    list_n = list(range(2,number))
    if number <= 1:
        return False
    elif number-1 > 2:
        for n in range(len(list_n)):
            list_n[n] = number % list_n[n]
        for n in list_n:
            if n == 0:
                return False
    return True



In [80]:
example_list,existing_ints=generate_example_list(is_prime_rule,generate_example_int,40,range_max=100)
print(f"example_list:{example_list}")
test_examples(example_list,existing_ints,is_prime_rule,attempts=100)


example_list:326 -> False, 425 -> False, 415 -> False, 28 -> True, 981 -> False, 609 -> False, 417 -> False, 615 -> False, 160 -> False, 266 -> False, 338 -> False, 115 -> False, 983 -> False, 357 -> False, 375 -> False, 994 -> False, 343 -> False, 309 -> False, 356 -> False, 244 -> False, 787 -> False, 315 -> False, 725 -> False, 512 -> False, 756 -> False, 852 -> False, 424 -> False, 668 -> False, 977 -> False, 762 -> False, 55 -> False, 444 -> False, 774 -> False, 444 -> False, 928 -> False, 41 -> True, 200 -> False, 502 -> False, 210 -> False, 413 -> False, 386 -> False, 479 -> False, 461 -> False, 845 -> False, 325 -> False, 85 -> False, 183 -> False, 495 -> False, 568 -> False, 734 -> False, 54 -> False, 119 -> False, 668 -> False, 439 -> False, 958 -> False, 617 -> False, 660 -> False, 147 -> False, 108 -> False, 331 -> False, 494 -> False, 820 -> False, 371 -> False, 293 -> False, 232 -> False, 120 -> False, 486 -> False, 752 -> False, 850 -> False, 394 -> False, 579 -> False, 

In [81]:
chat_completion=generate_explanations(example_list,existing_ints)
for response in chat_completion.choices:
       print(response.message.content)

generating response....
Example 216 correct
Explanations generated
Because 216 is not a power of 2.
The number 216 is not a multiple of 4, so it is not a multiple of any number other than 1 and itself. Therefore, it is not a prime number.
The number 216 is not a palindrome because it reads the same backwards as it does forwards.
216 is not a multiple of 4.
The number 216 does not meet the condition of having a digit sum greater than or equal to 10. The digit sum of 216 is 2 + 1 + 6 = 9, which is less than 10. Therefore, the statement "216 -> True" is incorrect.
